<a href="https://colab.research.google.com/github/Marlucben/Challenge-ONE-Data-Science-Telecom-X-parte-2/blob/main/Challenge_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carga de base de datos ##

In [1]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/Marlucben/Challenge-ONE-Data-Science-Telecom-X-parte-2/refs/heads/main/df_nuevo.json'
df = pd.read_json(url)
df.head()

ValueError: Trailing data